Imports

In [1]:
import helium
import selenium
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import datetime
import os

In [34]:
filename = 'additional_weather_data2.csv'
weather_dataframe_list = []
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 2, 25)

if os.path.isfile(filename):
    saved_weather_data = pd.read_csv(filename, low_memory=False, parse_dates=['Timestamp'])
    weather_dataframe_list = [saved_weather_data]
    last_datetime = saved_weather_data['Timestamp'].iloc[-1]
    start_date = datetime.date(year=last_datetime.year, month=last_datetime.month, day=last_datetime.day) + datetime.timedelta(days=1)
 
date_interval = (start_date, end_date)

In [22]:
base_url = 'https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/'

browser = helium.start_chrome(base_url, headless=True)
time.sleep(1)
helium.click(helium.S('//*[@id="wuSettings"]'))
time.sleep(1)
helium.click(helium.S('//*[@id="wuSettings-quick"]/div/a[2]'))
time.sleep(1)

In [35]:
current_date = date_interval[0]
numer_pattern = re.compile(r'(\d|\.|,)+')
date_format = '%I:%M %p'
helium.Config.implicit_wait_secs = 10


while current_date <= date_interval[1]:
    try:
        current_url = base_url + current_date.__str__()
        helium.go_to(current_url)
        time.sleep(1)
        helium.wait_until(helium.S('//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table').exists)
        
        soup = BeautifulSoup(browser.page_source, features='html.parser')
        weather_table = soup.find('table', attrs={'aria-labelledby':'History observation'})
        current_weather_dataframe = pd.read_html(str(weather_table))
        current_weather_dataframe = current_weather_dataframe[0]

        current_weather_dataframe.drop(['Wind', 'Wind Gust'], axis=1, inplace=True)
        current_weather_dataframe.dropna(inplace=True)

        for index, row in current_weather_dataframe.iterrows():
            weather_time = datetime.datetime.strptime(row['Time'], date_format)
            row['Time'] = datetime.datetime(year=current_date.year, month=current_date.month, day=current_date.day, hour=weather_time.hour, minute=0)

        current_weather_dataframe.drop_duplicates(['Time'], inplace=True)
        current_weather_dataframe.reset_index(drop=True, inplace=True)

        current_weather_dataframe['Temperature'] = pd.to_numeric(current_weather_dataframe['Temperature'].apply(lambda x: re.sub(r'(\s°C)', '', x)))
        current_weather_dataframe['Dew Point'] = pd.to_numeric(current_weather_dataframe['Dew Point'].apply(lambda x: re.sub(r'(\s°C)', '', x)))
        current_weather_dataframe['Humidity'] = pd.to_numeric(current_weather_dataframe['Humidity'].apply(lambda x: re.sub(r'(\s°%)', '', x)))
        current_weather_dataframe['Wind Speed'] = pd.to_numeric(current_weather_dataframe['Wind Speed'].apply(lambda x: re.sub(r'(\s°km/h)', '', x)))
        current_weather_dataframe['Pressure'] = pd.to_numeric(current_weather_dataframe['Pressure'].apply(lambda x: re.sub(r'(\s°hPa)', '', x.replace(',', ''))))
        current_weather_dataframe['Precip.'] = pd.to_numeric(current_weather_dataframe['Precip.'].apply(lambda x: re.sub(r'(\s°mm)', '', x)))
        current_weather_dataframe.rename(columns={"Time": "Timestamp"}, inplace=True)
        weather_dataframe_list.append(current_weather_dataframe)
    except selenium.common.exceptions.TimeoutException:
        pass
        
    current_date = current_date + datetime.timedelta(days=1)
    

In [36]:
weather_data = pd.concat(weather_dataframe_list)
weather_data

,Timestamp,Temperature,Dew Point,Humidity,Wind Speed,Pressure,Precip.,Condition
0,2023-01-01 01:00:00,12,11,93,6,1005.38,0.0,Light Rain
1,2023-01-01 02:00:00,12,10,86,6,1005.72,0.5,Light Rain
2,2023-01-01 03:00:00,12,9,83,19,1007.07,0.0,Light Rain
3,2023-01-01 04:00:00,12,9,80,17,1007.41,0.0,Light Rain
4,2023-01-01 05:00:00,12,8,77,9,1007.75,0.0,Light Rain
...,...,...,...,...,...,...,...,...
19,2023-02-25 20:00:00,1,-6,64,13,1021.28,0.0,Cloudy
20,2023-02-25 21:00:00,0,-6,66,11,1021.28,0.0,Mostly Cloudy
21,2023-02-25 22:00:00,-1,-5,72,11,1020.95,0.0,Partly Cloudy
22,2023-02-25 23:00:00,-1,-6,69,9,1020.61,0.0,Fair


In [37]:
weather_data.to_csv(filename, index=False)